In [2]:
pip install catboost

  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.5 MB 131.3 kB/s eta 0:13:01
   ---------------------------------------- 0.0/102.5 MB 196.9 kB/s eta 0:08:41
   ---------------------------------------- 0.1/102.5 MB 252.2 kB/s eta 0:06:47
   ---------------------------------------- 0.1/102.5 MB 450.6 kB/s eta 0:03:48
   ---------------------------------------- 0.2/102.5 MB 722.1 kB/s eta 0:02:22
   ---------------------------------------- 0.3/102.5 MB 884.2 kB/s eta 0:01:56
   ---------------------------------------- 0.6/102.5 MB 1.4 MB/s eta 0:01:14
   ---------------------

In [3]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [33]:
import numpy as np
import pandas as pd

# Load
df = pd.read_csv("health_dataset_with_issues.csv")

# Normalize column names (strip spaces, unify case, replace spaces with _)
df.columns = (
    df.columns.astype(str)
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)
)

print(df.shape)
df.head()


(1000, 22)


,Age,Sex,BMI,Smoker,AlcoholConsumption,PhysicalActivity,DietQuality,BloodPressure,GlucoseLevel,CholesterolLevel,...,Diabetes,Hypertension,Asthma,KidneyDisease,LiverDisease,Cancer,Obesity,Arthritis,COPD,MentalHealthIssue
0,56.0,Male,18.0,Yes,High,Active,Good,Hypertension,181.0,186.0,...,No,No,Yes,NaN,Yes,Yes,Yes,Yes,No,Yes
1,69.0,Male,33.3,No,Moderate,Sedentary,Poor,Hypertension,127.0,259.0,...,Yes,NaN,Yes,No,Yes,Yes,Yes,No,Yes,No
2,46.0,Female,19.8,No,Moderate,Sedentary,Good,NaN,147.0,219.0,...,No,No,Yes,Yes,Yes,No,Yes,No,Yes,No
3,32.0,Male,17.9,Yes,Moderate,Sedentary,Good,Normal,144.0,131.0,...,No,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No
4,60.0,Male,25.5,No,Moderate,Active,Good,Prehypertension,111.0,185.0,...,No,Yes,No,No,No,Yes,No,Yes,Yes,Yes


In [6]:
df.head()

,Age,Sex,BMI,Smoker,AlcoholConsumption,PhysicalActivity,DietQuality,BloodPressure,GlucoseLevel,CholesterolLevel,...,Diabetes,Hypertension,Asthma,KidneyDisease,LiverDisease,Cancer,Obesity,Arthritis,COPD,MentalHealthIssue
0,56.0,Male,18.0,Yes,High,Active,Good,Hypertension,181.0,186.0,...,No,No,Yes,NaN,Yes,Yes,Yes,Yes,No,Yes
1,69.0,Male,33.3,No,Moderate,Sedentary,Poor,Hypertension,127.0,259.0,...,Yes,NaN,Yes,No,Yes,Yes,Yes,No,Yes,No
2,46.0,Female,19.8,No,Moderate,Sedentary,Good,NaN,147.0,219.0,...,No,No,Yes,Yes,Yes,No,Yes,No,Yes,No
3,32.0,Male,17.9,Yes,Moderate,Sedentary,Good,Normal,144.0,131.0,...,No,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No
4,60.0,Male,25.5,No,Moderate,Active,Good,Prehypertension,111.0,185.0,...,No,Yes,No,No,No,Yes,No,Yes,Yes,Yes


In [34]:
TARGET = "HeartDisease"  # update here if your target has a different exact name

# If the column name differs slightly (e.g., 'Heart_Disease'), try to auto-detect:
if TARGET not in df.columns:
    candidates = [c for c in df.columns if c.lower().replace("_","") == "heartdisease"]
    if candidates:
        TARGET = candidates[0]
    else:
        raise ValueError("Target column not found. Please set TARGET to the exact column name.")

# Clean the target
df[TARGET] = (
    df[TARGET]
      .astype(str)
      .str.strip()
      .str.replace(r"\s+", " ", regex=True)
      .str.title()   # e.g., 'yes' -> 'Yes'
      .replace({"Y":"Yes", "N":"No"})
)

# Set unexpected values to NaN, then drop
valid = {"Yes", "No"}
df.loc[~df[TARGET].isin(valid), TARGET] = np.nan
before = len(df)
df = df.dropna(subset=[TARGET]).copy()
after = len(df)

print(f"Dropped {before - after} rows with missing/invalid target.")
print("Target value counts:\n", df[TARGET].value_counts(dropna=False))


Dropped 50 rows with missing/invalid target.
Target value counts:
 Yes    514
No     436
Name: HeartDisease, dtype: int64


In [35]:
# Convert obvious numeric-looking columns safely; keep others as object
for col in df.columns:
    if col == TARGET:
        continue
    # try forcing numeric if possible (non-convertible stay as object via 'ignore')
    if df[col].dtype == object:
        coerced = pd.to_numeric(df[col], errors="coerce")
        # If coercion succeeds for most values, keep numeric; else leave as object
        if coerced.notna().mean() >= 0.8:
            df[col] = coerced

# Split columns by dtype, excluding the target
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
cat_cols = df.select_dtypes(exclude=["number"]).columns.tolist()
if TARGET in cat_cols: cat_cols.remove(TARGET)
if TARGET in num_cols: num_cols.remove(TARGET)

print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)


Numeric columns: ['Age', 'BMI', 'GlucoseLevel', 'CholesterolLevel']
Categorical columns: ['Sex', 'Smoker', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'BloodPressure', 'FamilyHistory', 'Diabetes', 'Hypertension', 'Asthma', 'KidneyDisease', 'LiverDisease', 'Cancer', 'Obesity', 'Arthritis', 'COPD', 'MentalHealthIssue']


In [36]:
# Replace ±inf with NaN in numeric, then impute with median
for col in num_cols:
    col_vals = pd.to_numeric(df[col], errors="coerce")
    col_vals = col_vals.replace([np.inf, -np.inf], np.nan)
    med = col_vals.median()
    df[col] = col_vals.fillna(med)

# Impute categorical with mode
for col in cat_cols:
    # strip whitespace, empty -> NaN
    df[col] = df[col].astype(str).str.strip()
    df.loc[df[col].isin(["", "nan", "NaN", "None"]), col] = np.nan
    mode_val = df[col].mode(dropna=True)
    if len(mode_val):
        df[col] = df[col].fillna(mode_val.iloc[0])
    else:
        # if all missing, fill a sentinel
        df[col] = df[col].fillna("Unknown")

# Sanity check
print("Any NaN left in numeric?", df[num_cols].isna().sum().sum())
print("Any NaN left in categorical?", df[cat_cols].isna().sum().sum())


Any NaN left in numeric? 0
Any NaN left in categorical? 0


In [37]:
def iqr_clip(s: pd.Series, k: float = 1.5):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    low = q1 - k * iqr
    high = q3 + k * iqr
    return s.clip(lower=low, upper=high)

for col in num_cols:
    df[col] = iqr_clip(df[col].astype(float), k=1.5)

print("Outlier capping done.")


Outlier capping done.


In [38]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
X_cat = pd.DataFrame(
    ohe.fit_transform(df[cat_cols]),
    columns=ohe.get_feature_names_out(cat_cols),
    index=df.index
)

# Combine numeric and encoded categorical into a single features frame (X_ready)
X_num = df[num_cols].reset_index(drop=True)
X_cat = X_cat.reset_index(drop=True)
X_ready = pd.concat([X_num, X_cat], axis=1)

print("Encoded feature shape:", X_ready.shape)
X_ready.head()


Encoded feature shape: (950, 42)


,Age,BMI,GlucoseLevel,CholesterolLevel,Sex_Female,Sex_Male,Smoker_No,Smoker_Yes,AlcoholConsumption_High,AlcoholConsumption_Low,...,Cancer_No,Cancer_Yes,Obesity_No,Obesity_Yes,Arthritis_No,Arthritis_Yes,COPD_No,COPD_Yes,MentalHealthIssue_No,MentalHealthIssue_Yes
0,56.0,18.0,181.0,186.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
1,69.0,33.3,127.0,259.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,32.0,17.9,144.0,131.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,60.0,25.5,111.0,185.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,25.0,27.6,137.0,121.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [39]:
# Map target to 0/1
y = df[TARGET].map({"No": 0, "Yes": 1}).astype(int)
X = X_ready

print("NaNs in X:", int(X.isna().sum().sum()))
print("NaNs in y:", int(y.isna().sum()))
print("X shape:", X.shape, "| y shape:", y.shape)
print("y distribution:\n", y.value_counts(normalize=True).round(3))


NaNs in X: 0
NaNs in y: 0
X shape: (950, 42) | y shape: (950,)
y distribution:
 1    0.541
0    0.459
Name: HeartDisease, dtype: float64


In [18]:
# Map target to 0/1
y = df[TARGET].map({"No": 0, "Yes": 1}).astype(int)
X = X_ready

print("NaNs in X:", int(X.isna().sum().sum()))
print("NaNs in y:", int(y.isna().sum()))
print("X shape:", X.shape, "| y shape:", y.shape)
print("y distribution:\n", y.value_counts(normalize=True).round(3))


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)
print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")
print("\nClass distribution in Train:\n", y_train.value_counts(normalize=True).round(3))
print("\nClass distribution in Test:\n", y_test.value_counts(normalize=True).round(3))

Training set: (760, 42), Test set: (190, 42)

Class distribution in Train:
 1    0.541
0    0.459
Name: HeartDisease, dtype: float64

Class distribution in Test:
 1    0.542
0    0.458
Name: HeartDisease, dtype: float64


In [43]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate_classification(y_true, y_pred, y_proba):
    """
    Evaluate model performance with common classification metrics
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_proba)
    
    return accuracy, precision, recall, f1, auc


In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Models Dictionary
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
    "CatBoost": CatBoostClassifier(verbose=False, random_state=42)
}

# To store results
results = []

for name, model in models.items():
    print(f"Training {name}...")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Probabilities for ROC-AUC
    y_train_proba = model.predict_proba(X_train)[:, 1]
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    # Evaluate Train
    train_metrics = evaluate_classification(y_train, y_train_pred, y_train_proba)
    
    # Evaluate Test
    test_metrics = evaluate_classification(y_test, y_test_pred, y_test_proba)
    
    print(f"{name} Performance:")
    print(f" Train -> Accuracy: {train_metrics[0]:.4f} | Precision: {train_metrics[1]:.4f} | Recall: {train_metrics[2]:.4f} | F1: {train_metrics[3]:.4f} | AUC: {train_metrics[4]:.4f}")
    print(f" Test  -> Accuracy: {test_metrics[0]:.4f} | Precision: {test_metrics[1]:.4f} | Recall: {test_metrics[2]:.4f} | F1: {test_metrics[3]:.4f} | AUC: {test_metrics[4]:.4f}")
    print("="*60)
    
    # Append results
    results.append([name] + list(test_metrics))

# Convert results to DataFrame for easy comparison
import pandas as pd
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1 Score", "ROC-AUC"])
results_df = results_df.sort_values(by="ROC-AUC", ascending=False)
results_df


Training Logistic Regression...
Logistic Regression Performance:
 Train -> Accuracy: 0.5868 | Precision: 0.5896 | Recall: 0.7762 | F1: 0.6702 | AUC: 0.5823
 Test  -> Accuracy: 0.5000 | Precision: 0.5328 | Recall: 0.6311 | F1: 0.5778 | AUC: 0.4713
Training K-Nearest Neighbors...
K-Nearest Neighbors Performance:
 Train -> Accuracy: 0.7053 | Precision: 0.7082 | Recall: 0.7737 | F1: 0.7395 | AUC: 0.7556
 Test  -> Accuracy: 0.5158 | Precision: 0.5505 | Recall: 0.5825 | F1: 0.5660 | AUC: 0.5180
Training Decision Tree...
Decision Tree Performance:
 Train -> Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000 | AUC: 1.0000
 Test  -> Accuracy: 0.4579 | Precision: 0.5000 | Recall: 0.4951 | F1: 0.4976 | AUC: 0.4545
Training Random Forest...
Random Forest Performance:
 Train -> Accuracy: 1.0000 | Precision: 1.0000 | Recall: 1.0000 | F1: 1.0000 | AUC: 1.0000
 Test  -> Accuracy: 0.4579 | Precision: 0.5000 | Recall: 0.5534 | F1: 0.5253 | AUC: 0.4115
Training Gradient Boosting...
Gradie

,Model,Accuracy,Precision,Recall,F1 Score,ROC-AUC
1,K-Nearest Neighbors,0.515789,0.550459,0.582524,0.566038,0.517967
0,Logistic Regression,0.500000,0.532787,0.631068,0.577778,0.471264
2,Decision Tree,0.457895,0.500000,0.495146,0.497561,0.454469
5,AdaBoost,0.478947,0.515385,0.650485,0.575107,0.419931
3,Random Forest,0.457895,0.500000,0.553398,0.525346,0.411505
6,XGBoost,0.452632,0.495327,0.514563,0.504762,0.409887
7,CatBoost,0.431579,0.478992,0.553398,0.513514,0.383551
4,Gradient Boosting,0.431579,0.477477,0.514563,0.495327,0.379645
